In [1]:
# Cell 1: Imports
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path

In [2]:
# Cell 2: Load data (edit `path` if needed; Colab fallback included)
path = Path("Content/data/DIX.csv")  # <-- change if your file lives elsewhere

def load_dix_csv(p: Path) -> pd.DataFrame:
    if p.exists():
        df = pd.read_csv(p, parse_dates=["date"])
    else:
        # Colab-friendly fallback
        try:
            from google.colab import files
            uploaded = files.upload()
            fname = list(uploaded.keys())[0]
            df = pd.read_csv(fname, parse_dates=["date"])
        except Exception as e:
            raise FileNotFoundError(f"Could not find {p} and no upload provided.") from e
    df = df.sort_values("date").set_index("date")
    # Expected columns: price, dix, gex
    missing = {"price","dix","gex"} - set(df.columns.str.lower())
    if missing:
        raise ValueError(f"Missing expected columns: {missing}")
    # normalize column names just in case
    df = df.rename(columns={c:c.lower() for c in df.columns})
    return df[["price","dix","gex"]]

df = load_dix_csv(path)
df.head()


Saving DIX.csv to DIX.csv


,price,dix,gex
date,,,
2011-05-02,1361.219971,0.378842,1.897313e+09
2011-05-03,1356.619995,0.383411,1.859731e+09
2011-05-04,1347.319946,0.392122,1.717764e+09
2011-05-05,1335.099976,0.405457,1.361864e+09
2011-05-06,1340.199951,0.418649,1.490329e+09


In [3]:
# Cell 3: Build DEX + helpers
roll = 252  # ~1y trading days

df["ret"] = np.log(df["price"]).diff()
df["DEX"] = - df["gex"] * df["ret"]          # signed dealer-hedging flow proxy
df["DEX_abs"] = - df["gex"] * df["ret"].abs()# intensity proxy (direction-agnostic)

# rolling means & z-scores for scale invariance across regimes
for col in ["gex","DEX"]:
    mu = df[col].rolling(roll).mean()
    sd = df[col].rolling(roll).std()
    df[f"{col}_MA"] = mu
    df[f"{col}_z"]  = (df[col] - mu) / sd

# forward return windows for simple regime stats
H = [1, 5, 10]
for h in H:
    df[f"fwd_{h}d_ret"] = df["ret"].shift(-h).rolling(h).sum()

df = df.dropna()
df.tail()


,price,dix,gex,ret,DEX,DEX_abs,gex_MA,gex_z,DEX_MA,DEX_z,fwd_1d_ret,fwd_5d_ret,fwd_10d_ret
date,,,,,,,,,,,,,
2025-07-14,6268.56,0.485641,7.738057e+09,0.001406,-1.088292e+07,-1.088292e+07,4.669177e+09,1.258570,-1.187398e+07,0.024051,-0.003964,0.005891,0.019152
2025-07-15,6243.76,0.505412,7.181147e+09,-0.003964,2.846677e+07,-2.846677e+07,4.673933e+09,1.026653,-1.196998e+07,0.984249,0.003188,0.010493,0.020152
2025-07-16,6263.70,0.491125,7.489264e+09,0.003188,-2.387952e+07,-2.387952e+07,4.676793e+09,1.150291,-1.191697e+07,-0.291345,0.005359,0.015086,0.015713
2025-07-17,6297.36,0.505661,8.974995e+09,0.005359,-4.810089e+07,-4.810089e+07,4.685475e+09,1.746270,-1.203183e+07,-0.877155,-0.000091,0.010424,0.006652
2025-07-18,6296.79,0.488996,5.944323e+09,-0.000091,5.380695e+05,-5.380695e+05,4.678829e+09,0.516385,-1.183713e+07,0.301370,0.001398,0.014481,-0.009379


In [4]:
# Cell 4: Plotly figure (3 rows: Price+GEX z, DEX + 1y MA, Histogram)
specs = [[{"secondary_y": True}], [{}], [{}]]
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=False, vertical_spacing=0.12, specs=specs,
    subplot_titles=[
        "Price & GEX (Gamma Regimes)",
        "DEX (Dealer Hedging Flow) & 1-Yr Rolling Avg",
        "Distribution of Daily DEX"
    ],
)

# Row 1: Price (left) + GEX z-score (right)
fig.add_trace(go.Scatter(x=df.index, y=df["price"], name="Price"), row=1, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=df.index, y=df["gex_z"], name="GEX z-score"), row=1, col=1, secondary_y=True)

# Row 2: DEX + rolling average
fig.add_trace(go.Scatter(x=df.index, y=df["DEX"], name="DEX"), row=2, col=1)
fig.add_trace(go.Scatter(x=df.index, y=df["DEX_MA"], name="DEX 1-Yr MA", line=dict(dash="dash")), row=2, col=1)

# Row 3: Histogram of DEX
fig.add_trace(go.Histogram(x=df["DEX"], nbinsx=60, name="DEX"), row=3, col=1)
fig.add_vline(x=0, line_dash="dash", annotation_text="Zero", row=3, col=1)

# Axes & layout
fig.update_yaxes(title_text="Price", row=1, col=1, secondary_y=False)
fig.update_yaxes(title_text="GEX z", row=1, col=1, secondary_y=True)
fig.update_yaxes(title_text="DEX",  row=2, col=1)
fig.update_xaxes(title_text="DEX",  row=3, col=1)
fig.update_yaxes(title_text="Count",row=3, col=1)

fig.update_layout(
    template="plotly_dark",
    height=1100,
    margin=dict(t=90, b=60),
    showlegend=True,
    legend=dict(orientation="h", y=1.02, x=0.5, xanchor="center", yanchor="bottom"),
)
fig.show()


In [5]:
# Cell 5: Simple regime analytics (GEX sign & quintiles)
def regime_table(data: pd.DataFrame, label: str) -> pd.DataFrame:
    keep = ["ret"] + [f"fwd_{h}d_ret" for h in H]
    out = data[keep].agg(["count","mean","std"]).T
    out.columns = pd.MultiIndex.from_product([[label], out.columns])
    return out

# GEX sign regimes
sign_map = np.sign(df["gex"]).map({-1:"GEX<0", 0:"GEX≈0", 1:"GEX>0"})
tbl_sign = pd.concat([regime_table(df[sign_map==k], k) for k in ["GEX<0","GEX≈0","GEX>0"]], axis=1)

# GEX quintiles
q = pd.qcut(df["gex"], 5, labels=["Q1 Low","Q2","Q3","Q4","Q5 High"])
tbl_quint = pd.concat([regime_table(df[q==lab], lab) for lab in q.cat.categories], axis=1)

display(tbl_sign.round(4))
display(tbl_quint.round(4))


GEX<0                 GEX≈0               GEX>0                
             count    mean     std count    mean std   count    mean     std
ret          308.0 -0.0092  0.0203   1.0 -0.0089 NaN  3014.0  0.0014  0.0087
fwd_1d_ret   308.0  0.0007  0.0226   1.0 -0.0091 NaN  3014.0  0.0004  0.0087
fwd_5d_ret   308.0  0.0072  0.0399   1.0  0.0254 NaN  3014.0  0.0018  0.0195
fwd_10d_ret  308.0  0.0101  0.0552   1.0  0.0319 NaN  3014.0  0.0040  0.0268

Q1 Low                     Q2                     Q3          \
             count    mean     std  count    mean     std  count    mean   
ret          665.0 -0.0055  0.0170  664.0  0.0007  0.0100  665.0  0.0013   
fwd_1d_ret   665.0  0.0018  0.0177  664.0 -0.0003  0.0101  665.0 -0.0000   
fwd_5d_ret   665.0  0.0060  0.0331  664.0  0.0021  0.0219  665.0  0.0007   
fwd_10d_ret  665.0  0.0107  0.0441  664.0  0.0049  0.0300  665.0  0.0029   

                        Q4                 Q5 High                  
                std  count    mean     std   count    mean     std  
ret          0.0076  664.0  0.0025  0.0070   665.0  0.0033  0.0061  
fwd_1d_ret   0.0077  664.0  0.0003  0.0071   665.0  0.0005  0.0073  
fwd_5d_ret   0.0182  664.0  0.0016  0.0170   665.0  0.0011  0.0159  
fwd_10d_ret  0.0233  664.0  0.0021  0.0245   665.0  0.0024  0.0251

In [6]:
# Cell 6: (Optional) DIX filter panel — toggle thresholds here
DIX_LOW, DIX_HIGH = 40, 45   # % buckets; adjust to taste

lo = df[df["dix"] < DIX_LOW]
hi = df[df["dix"] > DIX_HIGH]

def quick_panel(a: pd.DataFrame, name: str):
    out = {
        "obs": len(a),
        "mean DEX": a["DEX"].mean(),
        "std DEX": a["DEX"].std(),
        "mean next-day ret": a["fwd_1d_ret"].mean(),
        "hit rate (up next-day)": (a["fwd_1d_ret"]>0).mean()
    }
    return pd.Series(out, name=name)

display(pd.concat([quick_panel(lo, f"DIX<{DIX_LOW}"), quick_panel(hi, f"DIX>{DIX_HIGH}")], axis=1).round(4))


,DIX<40,DIX>45
obs,3.323000e+03,0.0
mean DEX,-8.626501e+06,NaN
std DEX,2.976737e+07,NaN
mean next-day ret,5.000000e-04,NaN
hit rate (up next-day),5.426000e-01,NaN


In [7]:
# Cell A — Build the panel and plot a 2×2 bar dashboard
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Safety: define thresholds if not already present
DIX_LOW  = globals().get("DIX_LOW", 40)
DIX_HIGH = globals().get("DIX_HIGH", 45)

lo = df[df["dix"] < DIX_LOW]
hi = df[df["dix"] > DIX_HIGH]

def quick_panel(a: pd.DataFrame, name: str):
    return pd.Series({
        "obs": len(a),
        "mean_dex": a["DEX"].mean(),
        "std_dex": a["DEX"].std(),
        "mean_next1d_ret": a["fwd_1d_ret"].mean(),
        "hit_rate_up": (a["fwd_1d_ret"] > 0).mean(),
    }, name=name)

panel = pd.concat(
    [quick_panel(lo, f"DIX<{DIX_LOW}"), quick_panel(hi, f"DIX>{DIX_HIGH}")],
    axis=1
).T

# Plot
fig = make_subplots(
    rows=2, cols=2, vertical_spacing=0.18, horizontal_spacing=0.12,
    subplot_titles=[
        "Mean DEX", "Std DEX",
        "Mean Next-Day Return", "Hit Rate (Up Next-Day)"
    ]
)

# TL: mean DEX
fig.add_trace(go.Bar(x=panel.index, y=panel["mean_dex"], name="Mean DEX"), row=1, col=1)

# TR: std DEX
fig.add_trace(go.Bar(x=panel.index, y=panel["std_dex"], name="Std DEX"), row=1, col=2)

# BL: mean next-day return (as %)
fig.add_trace(
    go.Bar(x=panel.index, y=100*panel["mean_next1d_ret"], name="Mean 1d Ret (%)"),
    row=2, col=1
)

# BR: hit rate (as %)
fig.add_trace(
    go.Bar(x=panel.index, y=100*panel["hit_rate_up"], name="Hit Rate (%)"),
    row=2, col=2
)

fig.update_yaxes(title="DEX", row=1, col=1)
fig.update_yaxes(title="DEX", row=1, col=2)
fig.update_yaxes(title="%",   row=2, col=1)
fig.update_yaxes(title="%",   row=2, col=2)

fig.update_layout(
    template="plotly_dark",
    height=700,
    showlegend=False,
    margin=dict(t=80, b=40, l=60, r=30),
)
fig.show()

panel.round(4)


,obs,mean_dex,std_dex,mean_next1d_ret,hit_rate_up
DIX<40,3323.0,-8.626501e+06,2.976737e+07,0.0005,0.5426
DIX>45,0.0,NaN,NaN,NaN,NaN


In [8]:
# Cell B — Overlayed DEX distributions for DIX buckets
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=lo["DEX"], name=f"DIX<{DIX_LOW}", nbinsx=60, opacity=0.55
))
fig.add_trace(go.Histogram(
    x=hi["DEX"], name=f"DIX>{DIX_HIGH}", nbinsx=60, opacity=0.55
))

fig.update_layout(
    barmode="overlay",
    template="plotly_dark",
    title="DEX Distribution by DIX Regime",
    xaxis_title="DEX",
    yaxis_title="Count",
    margin=dict(t=60, b=40, l=60, r=30),
)
fig.add_vline(x=0, line_dash="dash", annotation_text="Zero")
fig.show()


Here’s a ready-to-paste **Markdown cell** for your notebook. It frames the analysis around **GEX**, defines **DEX**, explains the visuals, and gives interpretation tips + caveats.

---

# Dealer Positioning with GEX (and a DEX Flow Proxy)

## Why this notebook

We want a practical read on how dealer option positioning (via **GEX**) interacts with daily price moves to create stabilizing or amplifying hedging flows. The goal is the same spirit as IV–RV (implied vs realized) but for **gamma**: measure the *tension* between where dealers sit (GEX) and what the market just did (return), then visualize and summarize regimes.

---

## Key definitions

* **GEX (Gamma Exposure):** A proxy for the market’s aggregate options gamma that dealers are hedging.

  * **Sign:**

    * **GEX > 0 (long gamma):** Dealers hedge *against* moves (sell into strength, buy into weakness). Tends to **dampen** volatility.
    * **GEX < 0 (short gamma):** Dealers hedge *with* the move (buy into strength, sell into weakness). Tends to **amplify** volatility.
  * **Scale:** Changes over time with open interest/strikes; use **z-scores** or **quintiles** for cross-period comparisons.

* **DEX (Dealer Exposure “flow” proxy):**

  $$
  \textbf{DEX}_t = -\, \text{GEX}_t \times r_t,\quad\text{where } r_t=\Delta\ln(\text{Price}_t)
  $$

  Interpretation (sign = direction of hedge):

  * **DEX > 0:** Net **dealer buy** pressure that session.

    * Examples: (GEX>0 & down day) dealers buy the dip; (GEX<0 & up day) dealers chase higher.
  * **DEX < 0:** Net **dealer sell** pressure.

    * Examples: (GEX>0 & up day) dealers sell into strength; (GEX<0 & down day) dealers chase lower.
  * **DEX\_abs = − GEX × |r|** (optional): intensity proxy ignoring direction.

* **DIX (Dark Index, optional filter):** Used here as a conditioning variable to see if dark-pool participation shifts the DEX/return relationship.

---

## What the notebook builds

1. **DEX time series & 1-year rolling mean**

   * Highlights persistent buy/sell pressure regimes from dealer hedging.

2. **Price & GEX (z-score) panel**

   * Puts daily DEX in context of where gamma sits relative to its own history.

3. **DEX distribution (histogram)**

   * Shows skew/asymmetry in hedging pressure; a zero line makes sign balance obvious.

4. **Regime tables**

   * **By GEX sign** (GEX>0 vs GEX<0) and **by GEX quintiles**.
   * For each regime: sample size, next-day mean return (1/5/10d), volatility, and hit rates.

5. **DIX-conditioned dashboard (optional)**

   * Compare **DIX\<LOW** vs **DIX>HIGH** buckets on mean/std DEX, mean next-day return, and hit rate.
   * Overlayed histograms of DEX for both buckets.

---

## How to read the charts

* **Price & GEX (Gamma Regimes):**
  Watch for stretches where **GEX\_z** is strongly positive/negative. Persistent **GEX>0** regimes usually coincide with tighter realized vol and more mean-reverting microstructure; **GEX<0** patches align with trendy days and fatter tails.

* **DEX & 1-Yr MA:**

  * Sustained **DEX > 0** implies repeated buy-side hedging pressure (either stabilizing dips under long-gamma or momentum-chasing under short-gamma).
  * Cross the rolling mean to spot regime shifts in hedging dominance.

* **DEX Histogram:**

  * Right-skew → more frequent/stronger buy hedging; left-skew → more sell hedging.
  * Compare buckets (e.g., DIX high vs low) to see distributional shifts.

* **Regime tables:**

  * **GEX>0** should show milder forward returns dispersion on average; **GEX<0** tends to have wider distribution and bigger absolute moves.
  * Use **quintiles** to map monotonic effects (e.g., Q1→Q5).

---

## Practical uses

* **Risk stance:** Dial gross/net exposure based on GEX regime and recent DEX prints (e.g., reduce leverage when GEX flips negative and DEX clusters on the sell side).
* **Entry tactics:** Favor **fade** tactics in strong **GEX>0** regimes; favor **momentum/continuation** tactics in **GEX<0**.
* **Overlay filters:** Combine with **DIX** buckets, event windows (OPEX weeks), or VIX term-structure signals.

---

## Caveats

* **Units & vendors:** GEX levels are model-dependent; that’s why we use **z-scores/quintiles** for comparability.
* **DEX is a proxy:** It captures the *direction/size* implied by same-day hedging, not the full complexity of intraday hedger behavior.
* **Look-ahead hygiene:** We compute forward returns with proper shifting; keep that discipline if you extend.
* **Structural breaks:** Contract rolls, index membership changes, and regime shifts (e.g., 0DTE growth) can alter relationships—validate out-of-sample.

---

## Extensions you can add later

* **Horizon panel:** Plot mean/CI of forward returns for 1/5/10/20 days across GEX quintiles.
* **DEX\_abs regimes:** Use intensity buckets to study tail risk.
* **Event studies:** OPEX, CPI/FOMC days, end-of-month, and dealer roll windows.
* **Cross-assets:** Apply the same framework to indices with available gamma proxies.

---


GEX = Gamma Exposure — an estimate of the market-wide net gamma that dealers are carrying (inferred from listed options open interest). It answers: if the index moves 1%, how much delta will dealers need to trade to stay hedged?

How it’s built (conceptually):
For each option
𝑖
i: take its gamma
Γ
𝑖
Γ
i
​
  (per unit), scale by
𝑆
2
S
2
  to get “delta change per 1% move”, multiply by open interest and the contract multiplier, flip the sign to the dealer side (assuming customers are net long options), then sum across strikes and expiries:

GEX

≈

−
∑
𝑖
Γ
𝑖

𝑆
2

OI
𝑖

(
multiplier
)
GEX≈−
i
∑
​
 Γ
i
​
 S
2
 OI
i
​
 (multiplier)
Units are “delta (or notional) to trade per 1% move.”

Interpretation

GEX > 0 (dealers long gamma): dealers sell strength & buy weakness → tends to dampen moves / favor mean reversion.

GEX < 0 (dealers short gamma): dealers buy strength & sell weakness → can amplify moves / favor trend.

In the notebook we then use DEX = – GEX × return as a simple proxy for the actual daily hedging flow implied by that positioning.

In [10]:
import numpy as np, pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Assumes df already loaded from DIX.csv with columns: price, dix, gex
# If not, uncomment the two lines below:
# df = pd.read_csv("/mnt/data/DIX.csv", parse_dates=["date"]).sort_values("date").set_index("date")
# df = df.rename(columns=str.lower)[["price","dix","gex"]]

# --- Build returns, DEX, and z-scores ---
roll = 252
if "ret" not in df:
    df["ret"] = np.log(df["price"]).diff()

if "DEX" not in df:
    df["DEX"] = - df["gex"] * df["ret"]

def zscore(s, w=roll):
    mu = s.rolling(w).mean()
    sd = s.rolling(w).std()
    return (s - mu) / sd

df["gex_z"] = zscore(df["gex"], roll)
df["dex_z"] = zscore(df["DEX"], roll)
df["dex_z_ma"] = df["dex_z"].rolling(roll).mean()

# Choose a display window
start = pd.Timestamp("2023-01-01")
view = df.loc[df.index >= start].copy()

# --- Figure: Price (left), GEX_z + DEX_z + DEX_z 1y MA (right) ---
fig = make_subplots(specs=[[{"secondary_y": True}]],
                    subplot_titles=["Price & GEX (Gamma Regimes)"])

fig.add_trace(go.Scatter(x=view.index, y=view["gex_z"],
                         name="GEX z-score", mode="lines"), secondary_y=True)

fig.add_trace(go.Scatter(x=view.index, y=view["dex_z"],
                         name="DEX", mode="lines"), secondary_y=True)

fig.add_trace(go.Scatter(x=view.index, y=view["dex_z_ma"],
                         name="DEX 1-Yr MA", mode="lines",
                         line=dict(dash="dash")), secondary_y=True)

fig.update_yaxes(title_text="Price", secondary_y=False)
fig.update_yaxes(title_text="GEX / DEX (z)", secondary_y=True)

fig.update_layout(template="plotly_dark", height=420,
                  legend=dict(orientation="h", y=1.05, x=0.5,
                              xanchor="center", yanchor="bottom"),
                  margin=dict(t=70, b=40, l=60, r=60))
fig.show()
